Import Library

In [24]:
#general purpose packages
import numpy as np
import pandas as pd
import tensorflow as tf
import matplotlib.pyplot as plt
import seaborn as sns
import os

#data processing
import re, string
import emoji
import nltk

from sklearn import preprocessing
from imblearn.over_sampling import RandomOverSampler
from sklearn.model_selection import train_test_split

File Path

In [25]:
train_file_path = 'data/Index2/Raw/Corona_NLP_training.csv'
test_file_path = 'data/Index2/Raw/Corona_NLP_test.csv'

# train_file_path = 'data/Index2/Clean/cleaned_train_lite.csv'
# test_file_path = 'data/Index2/Clean/cleaned_train_lite.csv'

# train_file_path = 'data/Index3/Raw/sentimentdataset.csv'
# test_file_path = 'data/Index3/Raw/Corona_NLP_test.csv'
df = pd.read_csv(train_file_path,encoding='ISO-8859-1')
df_test = pd.read_csv(test_file_path)

Test

In [3]:
df.head()

,UserName,ScreenName,Location,TweetAt,OriginalTweet,Sentiment
0,3799,48751,London,16-03-2020,@MeNyrbie @Phil_Gahan @Chrisitv https://t.co/i...,Neutral
1,3800,48752,UK,16-03-2020,advice Talk to your neighbours family to excha...,Positive
2,3801,48753,Vagabonds,16-03-2020,Coronavirus Australia: Woolworths to give elde...,Positive
3,3802,48754,NaN,16-03-2020,My food stock is not the only one which is emp...,Positive
4,3803,48755,NaN,16-03-2020,"Me, ready to go at supermarket during the #COV...",Extremely Negative


In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 41157 entries, 0 to 41156
Data columns (total 6 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   UserName       41157 non-null  int64 
 1   ScreenName     41157 non-null  int64 
 2   Location       32567 non-null  object
 3   TweetAt        41157 non-null  object
 4   OriginalTweet  41157 non-null  object
 5   Sentiment      41157 non-null  object
dtypes: int64(2), object(4)
memory usage: 1.9+ MB


Handle duplicate tweets

In [26]:
df.drop_duplicates(subset='OriginalTweet',inplace=True)

# df.drop_duplicates(subset='Text',inplace=True)

In [6]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 41157 entries, 0 to 41156
Data columns (total 6 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   UserName       41157 non-null  int64 
 1   ScreenName     41157 non-null  int64 
 2   Location       32567 non-null  object
 3   TweetAt        41157 non-null  object
 4   OriginalTweet  41157 non-null  object
 5   Sentiment      41157 non-null  object
dtypes: int64(2), object(4)
memory usage: 1.9+ MB


Tweets Deep Cleaning

In [27]:
df = df[['OriginalTweet','Sentiment']]
df_test = df_test[['OriginalTweet','Sentiment']]

# df = df[['Text','Sentiment']]

In [28]:
def strip_emoji(text):
    return re.sub(emoji.get_emoji_regexp(), r"", text) #remove emoji

#Remove punctuations, links, mentions and \r\n new line characters
def strip_all_entities(text): 
    text = text.replace('\r', '').replace('\n', ' ').replace('\n', ' ').lower() #remove \n and \r and lowercase
    text = re.sub(r"(?:\@|https?\://)\S+", "", text) #remove links and mentions
    text = re.sub(r'[^\x00-\x7f]',r'', text) #remove non utf8/ascii characters such as '\x9a\x91\x97\x9a\x97'
    banned_list= string.punctuation + 'Ã'+'±'+'ã'+'¼'+'â'+'»'+'§'
    table = str.maketrans('', '', banned_list)
    text = text.translate(table)
    return text

#clean hashtags at the end of the sentence, and keep those in the middle of the sentence by removing just the # symbol
def clean_hashtags(tweet):
    new_tweet = " ".join(word.strip() for word in re.split('#(?!(?:hashtag)\b)[\w-]+(?=(?:\s+#[\w-]+)*\s*$)', tweet)) #remove last hashtags
    new_tweet2 = " ".join(word.strip() for word in re.split('#|_', new_tweet)) #remove hashtags symbol from words in the middle of the sentence
    return new_tweet2

#Filter special characters such as & and $ present in some words
def filter_chars(a):
    sent = []
    for word in a.split(' '):
        if ('$' in word) | ('&' in word):
            sent.append('')
        else:
            sent.append(word)
    return ' '.join(sent)

def remove_mult_spaces(text): # remove multiple spaces
    return re.sub("\s\s+" , " ", text)

In [7]:
texts_new = []
for t in df.OriginalTweet:
    texts_new.append(remove_mult_spaces(filter_chars(clean_hashtags(strip_all_entities(strip_emoji(t))))))

In [8]:
texts_new_test = []
for t in df_test.OriginalTweet:
    texts_new_test.append(remove_mult_spaces(filter_chars(clean_hashtags(strip_all_entities(strip_emoji(t))))))

In [29]:
df['sentence'] = texts_new
df_test['sentence'] = texts_new_test

In [15]:
df['sentence'].head()
print(df.iloc[1]['sentence'], df.iloc[1]['label'])

KeyError: 'label'

In [30]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 41157 entries, 0 to 41156
Data columns (total 3 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   OriginalTweet  41157 non-null  object
 1   Sentiment      41157 non-null  object
 2   sentence       41157 non-null  object
dtypes: object(3)
memory usage: 964.7+ KB


In [31]:
text_len = []
for text in df.sentence:
    tweet_len = len(text.split())
    text_len.append(tweet_len)

df['text_len'] = text_len

text_len_test = []
for text in df_test.sentence:
    tweet_len = len(text.split())
    text_len_test.append(tweet_len)
    
df_test['text_len'] = text_len_test

In [32]:
# Rename columns
df = df.rename(columns={'Sentiment': 'label'})
df_test = df_test.rename(columns={'Sentiment': 'label'})

In [33]:
#Remove naN

df = df.dropna(subset=['sentence'])
df_test = df_test.dropna(subset=['sentence'])
df['sentence'] = df['sentence'].astype(str)
df_test['sentence'] = df_test['sentence'].astype(str)

Convert label to integer

In [34]:
df['label'] = df['label'].map({'Extremely Negative':0,'Negative':0,'Neutral':1,'Positive':2,'Extremely Positive':2})
df_test['label'] = df_test['label'].map({'Extremely Negative':0,'Negative':0,'Neutral':1,'Positive':2,'Extremely Positive':2})


Sample down-size

In [38]:
print(df['label'].value_counts())
print(df_test['label'].value_counts())

label
2    17971
0    15335
1     7492
Name: count, dtype: int64
label
0    1632
2    1542
1     610
Name: count, dtype: int64


In [35]:
from sklearn.model_selection import train_test_split

# Định nghĩa tỷ lệ mẫu bạn muốn giữ (ví dụ: 20%)
sample_ratio = 0.9

# Sử dụng train_test_split với tham số stratify để giữ tỷ lệ nhãn
df, _ = train_test_split(df, test_size=1-sample_ratio, stratify=df['label'], random_state=42)
# df_test, _ = train_test_split(df_test, test_size=1-0.5, stratify=df_test['label'], random_state=42)
# Kiểm tra lại tỷ lệ nhãn trong tập dữ liệu mẫu
# print(df['label'].value_counts(normalize=True))
print(df['label'].value_counts())
print(df_test['label'].value_counts())


label
2    2338
0    1994
1    1000
Name: count, dtype: int64
label
0    816
2    773
1    310
Name: count, dtype: int64


In [15]:
# print(df['text_len'].value_counts())
# print(df_test['text_len'].value_counts())
df1 = df[df['sentence'].str.len() >= 5]
df2 = df[df['text_len'] > 5]
print(df.info())

print(df1.info())

print(df2.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 41157 entries, 0 to 41156
Data columns (total 4 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   OriginalTweet  41157 non-null  object
 1   label          41157 non-null  int64 
 2   sentence       41157 non-null  object
 3   text_len       41157 non-null  int64 
dtypes: int64(2), object(2)
memory usage: 1.3+ MB
None
<class 'pandas.core.frame.DataFrame'>
Index: 41126 entries, 0 to 41156
Data columns (total 4 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   OriginalTweet  41126 non-null  object
 1   label          41126 non-null  int64 
 2   sentence       41126 non-null  object
 3   text_len       41126 non-null  int64 
dtypes: int64(2), object(2)
memory usage: 1.6+ MB
None
<class 'pandas.core.frame.DataFrame'>
Index: 40798 entries, 1 to 41156
Data columns (total 4 columns):
 #   Column         Non-Null Count  Dtype 
---  ------      

In [32]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 41157 entries, 0 to 41156
Data columns (total 4 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   OriginalTweet  41157 non-null  object
 1   label          41157 non-null  int64 
 2   sentence       41157 non-null  object
 3   text_len       41157 non-null  int64 
dtypes: int64(2), object(2)
memory usage: 1.3+ MB


In [36]:
df = df[(df['text_len'] > 5 & df['text_len'] < 50)]
df_test = df_test[(df['text_len'] > 5 & df['text_len'] < 50)]

In [17]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 40798 entries, 1 to 41156
Data columns (total 4 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   OriginalTweet  40798 non-null  object
 1   label          40798 non-null  int64 
 2   sentence       40798 non-null  object
 3   text_len       40798 non-null  int64 
dtypes: int64(2), object(2)
memory usage: 1.6+ MB


Up sampling


In [19]:
from nltk.corpus import wordnet
# nltk.download('wordnet')
import random

def get_synonyms(word):
    synonyms = set()
    for syn in wordnet.synsets(word):
        for lemma in syn.lemmas():
            synonyms.add(lemma.name())
    return list(synonyms)

def synonym_replacement(sentence, n):
    words = sentence.split()
    new_words = words.copy()
    random_word_list = list(set(words))
    random.shuffle(random_word_list)
    num_replaced = 0
    for random_word in random_word_list:
        synonyms = get_synonyms(random_word)
        if len(synonyms) >= 1:
            synonym = random.choice(synonyms)
            new_words = [synonym if word == random_word else word for word in new_words]
            num_replaced += 1
        if num_replaced >= n:
            break
    return ' '.join(new_words)

df_up_sampling = df[df['label'] == 1]

df_up_sampling['sentence_sr'] = df_up_sampling['sentence'].apply(lambda x: synonym_replacement(x, 2))

df_up_sampling = df_up_sampling.sample(frac=0.25, random_state=42)

# Kết hợp lại với lớp đa số
# df_augmented = pd.concat([df, df_up_sampling[['sentence_sr', 'label']].rename(columns={'sentence_sr': 'sentence'})])



# df_up_sampling1.info()

C:\Users\Admin\AppData\Local\Temp\ipykernel_10160\577868193.py:30: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_up_sampling['sentence_sr'] = df_up_sampling['sentence'].apply(lambda x: synonym_replacement(x, 2))


In [22]:
df_augmented['label'].value_counts()

label
2    17971
0    15335
1    11365
Name: count, dtype: int64

In [23]:


df_selected = df_augmented[['sentence', 'label', 'text_len']]
save_train_file_path = 'data/Index2/Upsample/cleaned_train1.csv'

directory = os.path.dirname(save_train_file_path)

if not os.path.exists(directory):
    os.makedirs(directory)

    
df_selected.to_csv(save_train_file_path, index=False)

In [40]:
from sklearn.utils import resample
import pandas as pd

# Giả sử bạn có một DataFrame df với cột 'sentence' và 'label'
# Tách các mẫu của lớp thiểu số và lớp đa số
df_minority = df[df.label == 1]

# Tăng số lượng mẫu của lớp thiểu số bằng cách sao chép ngẫu nhiên
df_minority_upsampled = resample(df_minority, 
                                 replace=True,    # Lấy mẫu với thay thế
                                 n_samples=6000, # Số lượng mẫu mong muốn
                                 random_state=123) # Đặt seed để kết quả ổn định



# Kết hợp lại với lớp đa số
df_upsampled = pd.concat([df, df_minority_upsampled])

# Hiển thị kết quả
print(df_upsampled.label.value_counts())

# print(df_minority.head())
# print('----')
# print(df_minority_upsampled.head())

label
2    17971
0    15335
1    13492
Name: count, dtype: int64


In [21]:
df_augmented = pd.concat([df, df_up_sampling[['sentence_sr', 'label']].rename(columns={'sentence_sr': 'sentence'}), df_minority_upsampled])


In [41]:
df_selected = df_upsampled[['sentence', 'label', 'text_len']]
save_train_file_path = 'data/Index2/Upsample/cleaned_train4.csv'

directory = os.path.dirname(save_train_file_path)

if not os.path.exists(directory):
    os.makedirs(directory)

    
df_selected.to_csv(save_train_file_path, index=False)

In [56]:
df_selected = df[['sentence', 'label', 'text_len']]
df_test_selected = df_test[['sentence', 'label', 'text_len']]
save_train_file_path = 'data/Index2/Clean/cleaned_train.csv'
save_test_file_path = 'data/Index2/Clean/cleaned_test.csv'

directory = os.path.dirname(save_train_file_path)
directory1 = os.path.dirname(save_test_file_path)

if not os.path.exists(directory):
    os.makedirs(directory)
if not os.path.exists(directory1):
    os.makedirs(directory1)
    
df_selected.to_csv(save_train_file_path, index=False)
df_test_selected.to_csv(save_test_file_path, index=False)
